In [8]:
import pandas as pd
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from openpyxl.worksheet.cell_range import CellRange

CC = "79531599"
FIRST_YEAR = 2022
LAST_YEAR = 2024

In [2]:
df = pd.read_excel('../data/out/clean.xlsx')

In [3]:
df.isnull().sum()

NUM_CLASE          0
PROGRAMA           0
ASIGNATURA         0
TOTAL_HORAS        0
CICLO_LECTIVO      0
FECHA_INICIAL      0
FECHA_FINAL        0
CC_PROFESOR        0
NOMBRE_PROFESOR    0
TIPO_CONTRATO      0
FILE               0
YEAR               0
dtype: int64

In [4]:
df.dtypes

NUM_CLASE                  object
PROGRAMA                   object
ASIGNATURA                 object
TOTAL_HORAS               float64
CICLO_LECTIVO              object
FECHA_INICIAL      datetime64[ns]
FECHA_FINAL        datetime64[ns]
CC_PROFESOR                object
NOMBRE_PROFESOR            object
TIPO_CONTRATO              object
FILE                       object
YEAR                        int64
dtype: object

In [5]:
def most_frequent(x):
    return x.value_counts().idxmax()

def getNombreProfesor():
    filtered_df = df[df["CC_PROFESOR"] == CC]
    return filtered_df["NOMBRE_PROFESOR"].value_counts().idxmax()

In [9]:
result = {}

for CURRENT_YEAR in range(FIRST_YEAR, LAST_YEAR + 1):
    NOW = datetime.now().date()
    filtered_df = df[(df['CC_PROFESOR'] == CC) & (df['YEAR'] == CURRENT_YEAR) & (pd.to_datetime(df['FECHA_FINAL']).dt.date <= NOW)]
    if not filtered_df.empty:
        grouped_df = filtered_df.groupby(['NUM_CLASE', 'PROGRAMA', 'ASIGNATURA', 'CICLO_LECTIVO']).agg(
            TOTAL_HORAS=pd.NamedAgg(column='TOTAL_HORAS', aggfunc='sum'),
            FECHA_INICIAL=pd.NamedAgg(
                column='FECHA_INICIAL', aggfunc='min'),
            FECHA_FINAL=pd.NamedAgg(column='FECHA_FINAL', aggfunc='max'),
            TIPO_CONTRATO=pd.NamedAgg(
                column='TIPO_CONTRATO', aggfunc=most_frequent)
        ).reset_index()

        grouped_df['FECHA_INICIAL'] = grouped_df['FECHA_INICIAL'].dt.strftime('%d/%m/%Y')
        grouped_df['FECHA_FINAL'] = grouped_df['FECHA_FINAL'].dt.strftime('%d/%m/%Y')


        print(f"Results for year {CURRENT_YEAR}")
        result[f"{CURRENT_YEAR}"] = grouped_df
    else:
        print(f"No data for year {CURRENT_YEAR}")

Results for year 2022
Results for year 2023
Results for year 2024


In [10]:
nombreProfesor = getNombreProfesor()
excelFile = f"CERTIFICADO {nombreProfesor}.xlsx"
wb = Workbook()
ws = wb.active
ws.title = "Certificado"

header_font = Font(bold=True, color="FFFFFF")
subheader_font = Font(bold=True)
center_alignment = Alignment(horizontal='center')
thin_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))
blue_fill = PatternFill(start_color="062f52", end_color="062f52", fill_type="solid")

ws.append([nombreProfesor])
ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=8)
ws.cell(row=1, column=1).alignment = center_alignment
ws.cell(row=1, column=1).font = header_font

ws.append([])

for year, grouped_df in result.items():
    ws.append([f"Year: {year}"])
    ws.merge_cells(start_row=ws.max_row, start_column=1, end_row=ws.max_row, end_column=8)
    ws.cell(row=ws.max_row, column=1).alignment = center_alignment
    ws.cell(row=ws.max_row, column=1).font = header_font

    for r_idx, row in enumerate(dataframe_to_rows(grouped_df, index=False, header=True), 1):
        ws.append(row)
        if r_idx == 1:
            for cell in ws[ws.max_row]:
                cell.font = subheader_font

    ws.append([])

## Styling
for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
    for cell in row:
        if cell.value:
            cell.alignment = center_alignment
            cell.border = thin_border

## Merged Styling
for merged_cell_range in ws.merged_cells.ranges:
    cell_range = CellRange(merged_cell_range.coord)
    for row in ws.iter_rows(min_row=cell_range.min_row, max_row=cell_range.max_row,
                            min_col=cell_range.min_col, max_col=cell_range.max_col):
        for cell in row:
            cell.border = thin_border
            cell.fill = blue_fill
            cell.font = header_font

columnSize = [15, 45, 45, 20, 15, 15, 15, 15]

for index, width in enumerate(columnSize, start=1):
    col_letter = ws.cell(row=2, column=index).column_letter  # Obtener la letra de la columna desde la fila 3 (asegurarse de que no sea una celda fusionada)
    ws.column_dimensions[col_letter].width = width

wb.save(f"../data/certificates/{excelFile}")
print(f"Data saved to {excelFile}")

Data saved to CERTIFICADO GUTIERREZ SERRATO NEWMAN HERNAN.xlsx
